# Coursera Capstone Course

## About this repository
This repository represents the final assignment in the coursera certification course "IBM Data Science Professional Certificate" (See: https://www.coursera.org/professional-certificates/ibm-data-science).

## Preprocessing
To reduce clutter in the main notebook, I decided to split it in two components. This is the preprocessing notebook, where I use python to load data from the internet, cleaning it and saving it as a .csv file. This will allow me to keep the main component more readable.

In [1]:
import numpy as np
import pandas as pd

import requests
import codecs

import folium
import json

print("Finished importing.")

Finished importing.


### Loading data from the internet

In [2]:
# Links to sources
source_link_vvs = "https://www.openvvs.de/dataset/1f7ec4c1-b590-4751-a25b-57ef988c14b6/resource/d87d1f01-5c14-4d08-8452-e405a6472ab4/download/vvs_haltestellen.csv"

In [3]:
df_vvs = pd.read_csv(source_link_vvs, delimiter=";", encoding="iso-8859-1")

In [4]:
# Extract all possible ways of commotion (Verkehrsmittel = means of transport)
vm = list(df_vvs["Verkehrsmittel"].unique())

# This fields consists of multiple values seperated by a ';'
# We have to extract single words
vm_unique = set()

for y in vm:
    for x in y.split(";"):
        vm_unique.add(x) 

print(vm_unique)

{'Bus', 'Linientaxi', 'SEV-Bus', 'Seilbahn', 'Stadtbahn', 'Bürgerbus', 'Nachtbus', 'R-Bahn', 'Expressbus', 'Zahnradbahn', 'S-Bahn', 'Ruftaxi'}


In [5]:
# For this project I will only use connections using the so called 'S-Bahn' (a type 
# of train running only in a limited region)

# Extract entries where 
df_vvs = df_vvs[df_vvs["Verkehrsmittel"].str.match(".*S-Bahn.*")].reset_index()

print(df_vvs.shape)
df_vvs.head()

(84, 17)


,index,Nummer,Name,Name mit Ort,Globale ID,GKZ,Ort,Teilort,Landkreis,Tarifzonen,Verkehrsmittel,Linien (EFA),Linien (DIVA),Anzahl Linien,Betriebszweige,X-Koordinate,Y-Koordinate
0,13,34,Nürnberger Straße,Nürnberger Straße,de:08111:34,8111000,Stuttgart,Stuttgart,S,20,S-Bahn;R-Bahn;Nachtbus;Stadtbahn,"S2,S3,R2,R3,U1,N6","10002_,10003_,11002_,11003_,20001_,33006_",6,"10,11,20,33","9,234576905","48,80656049"
1,84,175,Leinfelden,Leinfelden,de:08116:175,8116078,Leinfeld.-Echterd.,Leinfelden,ES,38,Bus;S-Bahn;Stadtbahn,"S2,S3,S6,U5,35,38,E,82,86,826,826A","10002_,10003_,10006_,20005_,30035_,30038_,3003...",11,"10,20,30,31","9,142859698","48,69641917"
2,241,1003,Höfingen,Höfingen,de:08115:1003,8115028,Leonberg,Höfingen,BB,"36,46",S-Bahn,"S6,S60","10006_,10060_",2,10,"9,026180834","48,81503423"
3,243,1300,Sommerrain,Sommerrain,de:08111:1300,8111000,Stuttgart,Stuttgart,S,20,Bus;R-Bahn;S-Bahn,"S2,S3,R2,R3,54,58","10002_,10003_,11002_,11003_,30054_,30058_",6,"10,11,30","9,24834265","48,81524515"
4,244,1301,Malmsheim,Malmsheim,de:08115:1301,8115041,Renningen,Malmsheim,BB,56,S-Bahn,S6,10006_,1,10,"8,905105557","48,77183551"


In [6]:
# Print the names of the various train stations
df_vvs["Name"].values

array(['Nürnberger Straße', 'Leinfelden', 'Höfingen', 'Sommerrain',
       'Malmsheim', 'Rutesheim', 'Weil der Stadt', 'Bietigheim',
       'Kornwestheim', 'Neuwirtsh. (Porschep.)', 'Benningen (N)',
       'Freiberg (N)', 'Nellmersbach', 'Neustadt-Hohenacker',
       'Schwaikheim', 'Stetten-Beinstein', 'Geradstetten', 'Grunbach',
       'Weiler (R)', 'Winterbach', 'Altbach', 'Mettingen',
       'Oberesslingen', 'Zell', 'Filderstadt', 'Flughafen/Messe',
       'Oberaichen', 'Weilimdorf Bf', 'Süd', 'Magstadt', 'Nord',
       'Maichingen', 'Sindelfingen', 'Goldberg', 'Erdmannhausen',
       'Beutelsbach', 'Kirchheim (T)', 'Wernau (N)', 'Wendlingen (N)',
       'Ötlingen', 'Herrenberg', 'Ehningen', 'Gärtringen', 'Nufringen',
       'Rohr', 'Vaihingen', 'Universität', 'Österfeld', 'Schwabstraße',
       'Stadtmitte', 'Neckarpark', 'Untertürkheim', 'Obertürkheim',
       'Hauptbahnhof (oben)', 'Hauptbahnhof (tief)', 'Feuerbach',
       'Feuersee', 'Nordbahnhof', 'Bad Cannstatt', 'Zuffenhause

In [7]:
## Create a map to see if the location data seems reasonable

# Create a copy of the original dataframe
df_plot = df_vvs.loc[:, ["Name", "Y-Koordinate", "X-Koordinate"]]
df_plot.columns = ["Name", "Latitude", "Longitude"]

# Rectify some german conventions
formater = lambda x: x.replace(",", ".")
df_plot["Latitude"] = df_plot["Latitude"].apply(formater).astype(np.float64)
df_plot["Longitude"] = df_plot["Longitude"].apply(formater).astype(np.float64)

# Get mean lat and long as starting point
latitude = df_plot["Latitude"].mean()
longitude = df_plot["Longitude"].mean()

# Create the map and the labels
map = folium.Map(location = (latitude, longitude), zoom_start=10)
for _, (name, lat, long) in df_plot.iterrows():
    label = folium.Popup(name)
    
    folium.CircleMarker(
        location=(lat,long),
        popup=label,
        color="red",
        fill=True,
        radius=10,
        fill_opacity=0.2
    ).add_to(map)

map

In [8]:
print(df_vvs.columns)

Index(['index', 'Nummer', 'Name', 'Name mit Ort', 'Globale ID', 'GKZ', 'Ort',
       'Teilort', 'Landkreis', 'Tarifzonen', 'Verkehrsmittel', 'Linien (EFA)',
       'Linien (DIVA)', 'Anzahl Linien', 'Betriebszweige', 'X-Koordinate',
       'Y-Koordinate'],
      dtype='object')


In [9]:
# Drop unnecessary columns
df_vvs.drop(["index", "Nummer", "Name mit Ort", "Globale ID", "GKZ", "Landkreis", "Tarifzonen", "Linien (DIVA)","Anzahl Linien", "Betriebszweige"], axis=1, inplace=True)

df_vvs.head()

,Name,Ort,Teilort,Verkehrsmittel,Linien (EFA),X-Koordinate,Y-Koordinate
0,Nürnberger Straße,Stuttgart,Stuttgart,S-Bahn;R-Bahn;Nachtbus;Stadtbahn,"S2,S3,R2,R3,U1,N6","9,234576905","48,80656049"
1,Leinfelden,Leinfeld.-Echterd.,Leinfelden,Bus;S-Bahn;Stadtbahn,"S2,S3,S6,U5,35,38,E,82,86,826,826A","9,142859698","48,69641917"
2,Höfingen,Leonberg,Höfingen,S-Bahn,"S6,S60","9,026180834","48,81503423"
3,Sommerrain,Stuttgart,Stuttgart,Bus;R-Bahn;S-Bahn,"S2,S3,R2,R3,54,58","9,24834265","48,81524515"
4,Malmsheim,Renningen,Malmsheim,S-Bahn,S6,"8,905105557","48,77183551"


In [10]:
# Rename columns to english
df_vvs.columns = ["name", "place", "district", "type of transport", "lines", "longitude", "latitude"]
df_vvs.reset_index()

# Rectify some german conventions
formater = lambda x: x.replace(",", ".")
df_vvs["latitude"] = df_vvs["latitude"].apply(formater).astype(np.float64)
df_vvs["longitude"] = df_vvs["longitude"].apply(formater).astype(np.float64)

df_vvs.head()

,name,place,district,type of transport,lines,longitude,latitude
0,Nürnberger Straße,Stuttgart,Stuttgart,S-Bahn;R-Bahn;Nachtbus;Stadtbahn,"S2,S3,R2,R3,U1,N6",9.234577,48.806560
1,Leinfelden,Leinfeld.-Echterd.,Leinfelden,Bus;S-Bahn;Stadtbahn,"S2,S3,S6,U5,35,38,E,82,86,826,826A",9.142860,48.696419
2,Höfingen,Leonberg,Höfingen,S-Bahn,"S6,S60",9.026181,48.815034
3,Sommerrain,Stuttgart,Stuttgart,Bus;R-Bahn;S-Bahn,"S2,S3,R2,R3,54,58",9.248343,48.815245
4,Malmsheim,Renningen,Malmsheim,S-Bahn,S6,8.905106,48.771836


In [11]:
# Write data to a file
df_vvs.to_csv("data/stations.csv", encoding="utf-8", index=False)

### Loading data from Foursquare

In [12]:
data = pd.read_json(".private_data.json")["FOURSQUARE"]

# Authentification data
CLIENT_ID = data["ID"]
CLIENT_SECRET = data["SECRET"]

# Parameters
LIMIT = 10
RADIUS = 500 # 500 meters within range of location
VERSION = "20201115"

# Code url fragments
url_base = "https://api.foursquare.com/v2/venues/search?"
url_client = f"&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}"
url_parameters = f"&radius={RADIUS}&limit={LIMIT}&v={VERSION}"
url_payload = "&ll={},{}"
assemble = lambda ll: url_base + url_client + url_parameters + url_payload.format(ll[0], ll[1])


# Iterate through all stations and query the API for near venues
print("Start querying the API...")
results = {}
for _, (name, _, _, _, _, long, lat) in df_vvs.iterrows():
    
    # Assemble url
    url = assemble((lat, long))
    # print(f"Send query '{url}' to foursquare API.")
    
    # Query API and save the result for further processing
    result = requests.get(url).text
    results[name] = result
print("Done.")

# Delete private fields so they are not accesible from outside this cell
del(CLIENT_ID, CLIENT_SECRET, url_client, assemble)

Start querying the API...
Done.


In [13]:
print(f"Number of results: {len(results)} for {df_vvs.shape[0]} stations.")

# Print part of the string to see if everything seems correct
print(f"Sample:\n{results[list(results.keys())[0]][:500]}...")

Number of results: 84 for 84 stations.
Sample:
{"meta":{"code":200,"requestId":"5fb1d23671cf8433224bc3a1"},"response":{"venues":[{"id":"4c694a008e7c0f47862cdf0f","name":"S+U Stuttgart Nürnberger Straße","location":{"address":"Nürnberger Str.","crossStreet":"Obere Waiblinger Str.","lat":48.804362915308346,"lng":9.235662341750006,"labeledLatLngs":[{"label":"display","lat":48.804362915308346,"lng":9.235662341750006}],"distance":257,"postalCode":"70374","cc":"DE","city":"Stuttgart","state":"Baden-Württemberg","country":"Deutschland","formattedAd...


In [14]:
# Print a sample to get some feeeling about the layout of the documents
index = list(results.keys())[0]
result = results[index]

result = json.loads(result)["response"]["venues"]

print(result[0].keys())
print()
print(result[0]["categories"])

dict_keys(['id', 'name', 'location', 'categories', 'referralId', 'hasPerk'])

[{'id': '4bf58dd8d48988d1fc931735', 'name': 'Light Rail Station', 'pluralName': 'Light Rail Stations', 'shortName': 'Light Rail', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/lightrail_', 'suffix': '.png'}, 'primary': True}]


In [15]:
def extract_values(sname, venue_json):
    venue_json = json.loads(venue_json)["response"]["venues"]

    # Iterate through all venues in the json object
    venues = []
    for venue in venue_json:
        id = venue["id"]
        name = venue["name"]
        address = venue["location"].get("address")
        latitude = venue["location"]["lat"]
        longitude = venue["location"]["lng"]
        categories = [x["name"] for x in venue["categories"]]
    
        venues.append([sname, id, name, address, latitude, longitude, ",".join(categories)])
    
    return venues
        
def extract_all(venue_jsons):
    # Extract values from each response
    extracts = [extract_values(x[0], x[1]) for x in venue_jsons.items()]
    
    # Flatten the result
    res = [y for x in extracts for y in x]
    
    columns = [
        "station name",
        "venue id",
        "venue name",
        "venue address",
        "venue latitude",
        "venue longitude",
        "venue categories"
    ]
    
    return pd.DataFrame(data=res, columns=columns)

In [16]:
df_venues = extract_all(results)

df_venues.head()

,station name,venue id,venue name,venue address,venue latitude,venue longitude,venue categories
0,Nürnberger Straße,4c694a008e7c0f47862cdf0f,S+U Stuttgart Nürnberger Straße,Nürnberger Str.,48.804363,9.235662,Light Rail Station
1,Nürnberger Straße,4f3668a1e4b0aa62a107c15d,Agip Service Station,Nürnberger Straße 18,48.805813,9.233371,Gas Station
2,Nürnberger Straße,4c62a9097c9def3b4a81d31c,TOTAL Tankstelle,Nuernberger Str. 9,48.806046,9.232760,Gas Station
3,Nürnberger Straße,4c403e24e26920a1a48c5ce7,Amadeus Frisör,Nauheimerstrasse,48.805436,9.227753,
4,Nürnberger Straße,518920da498e0c6939ea0b60,Neckar Käpt'n,None,48.808496,9.234980,Harbor / Marina


In [17]:
# Replace all unknown tokens with a unknown string
df_venues[pd.isna(df_venues["venue categories"])]["venue categories"] = "Unknown"
df_venues["venue categories"].replace("", "Unknown", inplace = True)

df_venues.head()

,station name,venue id,venue name,venue address,venue latitude,venue longitude,venue categories
0,Nürnberger Straße,4c694a008e7c0f47862cdf0f,S+U Stuttgart Nürnberger Straße,Nürnberger Str.,48.804363,9.235662,Light Rail Station
1,Nürnberger Straße,4f3668a1e4b0aa62a107c15d,Agip Service Station,Nürnberger Straße 18,48.805813,9.233371,Gas Station
2,Nürnberger Straße,4c62a9097c9def3b4a81d31c,TOTAL Tankstelle,Nuernberger Str. 9,48.806046,9.232760,Gas Station
3,Nürnberger Straße,4c403e24e26920a1a48c5ce7,Amadeus Frisör,Nauheimerstrasse,48.805436,9.227753,Unknown
4,Nürnberger Straße,518920da498e0c6939ea0b60,Neckar Käpt'n,None,48.808496,9.234980,Harbor / Marina


In [18]:
# Write data to a file
df_venues.to_csv("data/venues.csv", encoding="utf-8", index=False)

### The End
To see what we can do with this data, refer the other notebook in this folder.